In [1]:
from selenium import webdriver

driver = webdriver.Chrome()
# 창끄면 핸들러 잃어버림
# driver.close()

In [2]:
driver.get("https://news.naver.com/")


In [27]:
######## 1번 방식 링크들어가서 제목, 내용 한번에 스크래핑
## 원래 스크래핑은 이렇게 하는것이다 이런식으로 해야함

In [44]:

con = sqlite3.connect("news_no_1.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table1_id INTEGER NOT NULL,
        content TEXT NOT NULL
    );

""")
con.commit()

driver.get("https://news.naver.com")

for _ in driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a"):
    if len(_.get_attribute("href"))>1:
        cur.execute("INSERT INTO table1(title,url) VALUES(?,?)",[_.text, _.get_attribute("href")])
        con.commit()
        
cur.execute("SELECT * FROM table1") 
for _ in cur.fetchall():
    driver.get(_[2])
    
    cur.execute("INSERT INTO table2(table1_id, content) VALUES(?,?)",[_[0],driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()

In [ ]:
####### 2번 방식 제목갔다가 본문갔다가 왔다갔다하면서 데이터 스크래핑
####### ajax 동기식으로 하는거 연습해볼려고 해본거다.

In [ ]:
import sqlite3
con = sqlite3.connect("news_no_2.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        url TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table1_id INTEGER NOT NULL,
        content TEXT NOT NULL
    );

""")
con.commit()

In [36]:
# 셀레니움 어떤 타입으로 가져오는지
N = 0
for _ in driver.find_elements_by_css_selector(".hdline_news  .hdline_article_tit > a, .com_list li > a") :
    if len(_.get_attribute("href")) > 1:
      #  print(_.get_attribute("href"))
        print(_.text)
        N = N + 1
      #  print (len(_.get_attribute("href")))


"다스 소송비, MB에 자금 지원 의미"…이학수 또 증언
임협 재개 불구 파업 가결로 현대중 노사 '가시밭길'
민주평화당 분당 수순…박지원 의원, 사실상 '신당 선언'
조희연, 자사고 폐지 ‘속도전’… 대책은 재탕
'성폭행 피소' 김준기, 치료 핑계 美 체류…신병 인도 요청
[팩트체크K] “강제 동원 판결, 노무현 정부 결정 뒤집었다”…사실은?
“일본판 기사에 ‘혐한’ 제목”…靑, 조선·중앙 공개 비판
[포커스] 한국당 '총선 필패론' 위기감…2030에 '비호감' 낙인
2~3주내 협상 시작한다더니…트럼프 "시간은 본질 아냐"
청와대, 조선·중앙일보 향해 “이게 국민 목소리인가”…조국 “매국적”
불산, 수입선 다변화·국산 개발…‘탈일본’ 속도
이재용 "스마트폰·TV도 비상계획 마련"…소재 탈일본 가능?
종업원에게도 지분 나눠줬던 술집, 은밀하게 '세금 회피'
택시 서비스·요금 다양화 예고…"진입장벽 높여" 반발도
자고 있는데 아이폰 '펑'…"조사 결과 못 알려줘"
대구 치맥페스티벌 열기 '후끈'…닭 튀김 로봇도 '눈길'
하룻밤새 무너진 사무실…CCTV에 찍힌 '재개발 폭력사태'
1600만원 넣어놓은 '보물 밥솥'을…경찰 도움으로 되찾아
검찰 "증인 1%만 신문"…'양승태 조기석방 반대' 의견서
대구 스크린골프장서 방화 추정 불…2명 부상 1명 위독(종합)
여성이 남성보다 알츠하이머 환자 많은 이유 찾았다…美 연구팀 결과
[뉴있저] "인생은 지금부터!"...77세 모델 최순화
호남·경남 150mm 큰비…주말 태풍 영향권
타다 “진입장벽 더 높아졌다”-택시 “개편안 큰 틀은 환영”
리블린 대통령, 여의도순복음교회 기도회 참석…"양국가교 노력"
日 지한파 오쿠조노 교수 “압류자산 현금화 멈추는 것이 日 요구의 마지노선”
규제의 역설…반도체 가격·주가 동반 상승
“한국, 불화수소 중국에 대량 주문”…일본 언론 ‘화들짝’ 보도
아베, 야유 막으려 거리유세에 자민당원 동원까지
마음 급한 韓 반도체회사..."中 불화수소 주문"
'부활의 날개짓' 싸이월드, 기대감 UP ↑
당

In [38]:
for i in range(N):
    driver.find_element_by_css_selector("#lnb > ul > li:first-child > a").click()
    news = [_ for _ in driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a") \
            if len(_.get_attribute("href")) > 1][i]
    
    cur.execute("INSERT INTO table1(title, url)  VALUES(? ,?)", [news.text, news.get_attribute("href")])
    con.commit()
    
    cur.execute("SELECT id FROM table1 WHERE url=?",[news.get_attribute("href")])
    newsID = cur.fetchone()[0]
    

    news.click()
    
    cur.execute("INSERT INTO table2(table1_id, content) VALUES(?, ?)", [newsID, driver.find_element_by_css_selector("#articleBodyContents").text])
    con.commit()

In [ ]:
from urllib import *
from html import unescape
import requests
import json 
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
import requests
import time
import bs4
from bs4 import BeautifulSoup
import re
import time


def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url, params= param, data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")
    ## 정규식 re.search(".+?\.tistory.com", "http:\\adam24eve.tistory.com/") #짧게매칭되는거 ? 레이지 

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
   # if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
   #     return []
    
   
   # seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
  #  print("-=================================================")
   # print(seed)
  #  print("-=================================================")
   # html = download("get",seed)
   # dom = BeautifulSoup(html.text, "lxml")
    

    return  [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if  _.has_attr("href") and len(_["href"]) >3 ]

##and checkBlog(requests.compat.urljoin(seed, _["href"]))




